In [ ]:
!git clone https://github.com/joseigmartinez/superstore-dashboard.git

In [ ]:
!ls

In [ ]:
%cd superstore-dashboard

In [ ]:
!ls

In [ ]:
!python 01_data_loading.py

In [ ]:
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, Dropdown
from IPython.display import display, HTML, clear_output
from pathlib import Path
from plotly.subplots import make_subplots

In [ ]:
# 1. CONEXIÓN Y CARGA DE DATOS

DB_FILE = 'superstore.db'
try:
    conn = sqlite3.connect(DB_FILE)
    print(f"Conexión exitosa a {DB_FILE}. Cargando datos...")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}. Asegúrate de ejecutar 01_data_loading.py primero.")
    exit() # Detiene la ejecución si no puede conectar

#Función auxiliar para corregir la coma decimal después de la carga
def convert_to_float(df, columns):
    for col in columns:
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Consulta SQL con JOIN completo para el dashboard (CORREGIDA)
query = """
SELECT
    O.OrderID, O.OrderDate, O.YearOrderDate, O.Sales, O.Quantity, O.Discount, O.Profit,
    P.ProductName, P.Subcategory, CAT.Category,
    S.ShipDate, SM.ShipMode,
    SEG.Segment, C.Country, C.State, C.PostalCode, REG.Region, CTY.City
FROM
    Ordenes O
LEFT JOIN Productos P ON O.ProductID = P.ProductID
LEFT JOIN Category CAT ON P.CategoryID = CAT.CategoryID
LEFT JOIN Ship S ON O.OrderID = S.OrderID
LEFT JOIN ShipMode SM ON S.ShipModeID = SM.ShipModeID
LEFT JOIN Clientes C ON O.CustomerID = C.CustomerID
LEFT JOIN Segment SEG ON C.SegmentID = SEG.SegmentID
LEFT JOIN City CTY ON C.CityID = CTY.CityID
LEFT JOIN Region REG ON C.RegionID = REG.RegionID;
"""
df = pd.read_sql_query(query, conn)
conn.close()

# Aplicar la corrección numérica
#df = convert_to_float(df, ['Sales', 'Profit', 'Discount'])
print(f"Datos cargados. Total de {len(df)} registros listos.")

In [ ]:
# --- VERIFICACIÓN DEL JOIN COMPLETO ---
print("\n--- Vista Previa del DataFrame Final (df.head()) ---")
display(df.head())
# --- VERIFICACIÓN DEL CORRECTO TIPO DE DATOS ---
print("\n--- Tipos de Datos (df.dtypes) ---")
display(df.dtypes)
# -------------------------------------

In [ ]:
# --- 2. IMPLEMENTACIÓN DE KPIs ---

# 2.1 KPI: Cantidad de órdenes totales, sales y profit
print("\n--- 1. KPI: Métricas Globales ---")
total_orders = df['OrderID'].nunique()
total_sales = df['Sales'].sum()
total_profit = df['Profit'].sum()

display(HTML(f"""
    <div style='display: flex; justify-content: space-around; text-align: center; border: 1px solid #ddd; padding: 15px; border-radius: 8px;'>
        <div>
            <h4>Órdenes Totales</h4>
            <p style='font-size: 24px; color: #007bff;'>{total_orders:,}</p>
        </div>
        <div>
            <h4>Facturación</h4>
            <p style='font-size: 24px; color: #28a745;'>${total_sales:,.0f}</p>
        </div>
        <div>
            <h4>Ganancias</h4>
            <p style='font-size: 24px; color: #ffc107;'>${total_profit:,.0f}</p>
        </div>
    </div>
"""))

In [ ]:
# 2.2 Gráfico de línea: Evolución de profit por año de orderdate
print("\n--- 2. Evolución de Profit por Año ---")
profit_by_year = df.groupby('YearOrderDate')['Profit'].sum().reset_index()

# Convertir el año a string/categórico para forzar a Plotly a mostrar un solo tick por año.
profit_by_year['YearOrderDate'] = profit_by_year['YearOrderDate'].astype(str)

fig = px.line(
    profit_by_year, x='YearOrderDate', y='Profit',
    title='Evolución del Beneficio (Profit) por Año de Orden',
    markers=True, labels={'YearOrderDate': 'Año', 'Profit': 'Beneficio ($)'},
    line_shape='spline', color_discrete_sequence=['#ff4500']
)
fig.update_layout(xaxis=dict(tickformat='d'))
fig.show()

In [ ]:
# 2.3 Gráfico de barras: Ventas por año
print("\n--- 3. Gráfico de Barras: Ventas Totales por Año ---")

# 1. Agrupación y Suma de Ventas
sales_by_year = df.groupby('YearOrderDate')['Sales'].sum().reset_index()

# 2. Corrección del Eje X: Convertir el año a string para evitar repetición
sales_by_year['YearOrderDate'] = sales_by_year['YearOrderDate'].astype(str)

# 3. Creación del Gráfico
fig = px.bar(
    sales_by_year,
    x='YearOrderDate',
    y='Sales',
    title='Ventas Totales por Año',
    labels={'YearOrderDate': 'Año', 'Sales': 'Ventas Totales ($)'},
    color_discrete_sequence=['#4c78a8'] # Color azul sólido
)

# 4. Visualización simple
fig.show()

In [ ]:
# 2.4 Gráfico de torta: Porcentaje de órdenes por categoria
print("\n--- 4. Porcentaje de Órdenes por Categoría ---")
orders_by_category = df.groupby('Category')['OrderID'].nunique().reset_index().rename(columns={'OrderID': 'TotalOrders'})
fig = px.pie(
    orders_by_category, names='Category', values='TotalOrders',
    title='Distribución Porcentual de Órdenes por Categoría', hole=.3,
    color_discrete_sequence=['#4c78a8']
)
fig.update_traces(textinfo='percent+label', marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# 2.5 Gráficos de torta: Órdenes y ganancias por región
print("\n--- 5. Distribución de Órdenes y Ganancias por Región ---")

# 1. Agrupación de métricas por Región
region_metrics = df.groupby('Region').agg(
    TotalOrders=('OrderID', 'nunique'),
    TotalProfit=('Profit', 'sum')
).reset_index()

# 2. Gráfico de Órdenes por Región
fig_orders = px.pie(
    region_metrics,
    names='Region',
    values='TotalOrders',
    title='Distribución de Órdenes por Región',
    hole=.4,
    color_discrete_sequence=['#4c78a8']
)
fig_orders.update_traces(textinfo='percent+label')

fig_orders.update_layout(showlegend=False)

# 3. Gráfico de Ganancias (Profit) por Región
fig_profit = px.pie(
    region_metrics,
    names='Region',
    values='TotalProfit',
    title='Distribución de Ganancias (Profit) por Región',
    hole=.4,
    color_discrete_sequence=['#4c78a8']
)
fig_profit.update_traces(textinfo='percent+label')

fig_profit.update_layout(showlegend=False)

# 4. Visualización de ambos gráficos
fig_orders.show()
fig_profit.show()

In [ ]:
# 2.6 Gráfico de tortas: Porcentaje de órdenes por ShipMode
print("\n--- 6. Porcentaje de Órdenes por Modo de Envío ---")
orders_by_shipmode = df.groupby('ShipMode')['OrderID'].nunique().reset_index().rename(columns={'OrderID': 'TotalOrders'})
fig = px.pie(
    orders_by_shipmode, names='ShipMode', values='TotalOrders',
    title='Distribución Porcentual de Órdenes por Modo de Envío', hole=.3,
    color_discrete_sequence=['#4c78a8']
)
fig.update_traces(textinfo='percent+label')
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# 2.7 Gráfico de lineas: Evolución de profit a través de los años
print("\n--- 7. Evolución de Profit por Categoría ---")
profit_by_year_category = df.groupby(['YearOrderDate', 'Category'])['Profit'].sum().reset_index()
fig = px.line(
    profit_by_year_category, x='YearOrderDate', y='Profit', color='Category',
    title='Evolución Anual de Profit por Categoría', markers=True,
    labels={'YearOrderDate': 'Año de la Orden', 'Profit': 'Beneficio ($)'}
)
fig.update_layout(xaxis=dict(tickformat='d'))
fig.show()

In [ ]:


# --- 1. KPIS ---

total_orders = df['OrderID'].nunique()
total_sales = df['Sales'].sum()
total_profit = df['Profit'].sum()


# 2. Gráfico de Línea: Evolución de Profit por Año (2.2)
profit_by_year = df.groupby('YearOrderDate')['Profit'].sum().reset_index()
profit_by_year['YearOrderDate'] = profit_by_year['YearOrderDate'].astype(str)
fig_22 = px.line(profit_by_year, x='YearOrderDate', y='Profit', markers=True, color_discrete_sequence=['#4c78a8'])
fig_22.update_traces(showlegend=False)

# 3. Gráfico de Barras: Ventas por Año (2.3)
sales_by_year = df.groupby('YearOrderDate')['Sales'].sum().reset_index()
sales_by_year['YearOrderDate'] = sales_by_year['YearOrderDate'].astype(str)
fig_23 = px.bar(sales_by_year, x='YearOrderDate', y='Sales', color_discrete_sequence=['#4c78a8'])
fig_23.update_layout(showlegend=False)
fig_23.update_traces(showlegend=False)

# 4. Gráfico de Torta: Órdenes por Categoría (2.4)
orders_by_category = df.groupby('Category')['OrderID'].nunique().reset_index().rename(columns={'OrderID': 'TotalOrders'})
fig_24 = px.pie(orders_by_category, names='Category', values='TotalOrders', hole=.3, color_discrete_sequence=['#4c78a8'])
fig_24.update_traces(textinfo='percent+label').update_layout(showlegend=False)
fig_24.update_traces(showlegend=False)

# 5. Gráficos de Torta: Órdenes y Profit por Región (2.5)
region_metrics = df.groupby('Region').agg(TotalOrders=('OrderID', 'nunique'), TotalProfit=('Profit', 'sum')).reset_index()
fig_25_orders = px.pie(region_metrics, names='Region', values='TotalOrders', hole=.4, color_discrete_sequence=['#4c78a8'])
fig_25_orders.update_traces(textinfo='percent+label').update_layout(showlegend=False)
fig_25_profit = px.pie(region_metrics, names='Region', values='TotalProfit', hole=.4, color_discrete_sequence=['#4c78a8'])
fig_25_profit.update_traces(textinfo='percent+label').update_layout(showlegend=False)
fig_25_profit.update_traces(showlegend=False)

# 6. Gráfico de Torta: Órdenes por ShipMode (2.6)
orders_by_shipmode = df.groupby('ShipMode')['OrderID'].nunique().reset_index().rename(columns={'OrderID': 'TotalOrders'})
fig_26 = px.pie(orders_by_shipmode, names='ShipMode', values='TotalOrders', hole=.3, color_discrete_sequence=['#4c78a8'])
fig_26.update_traces(textinfo='percent+label').update_layout(showlegend=False)
fig_26.update_traces(showlegend=False)

# 7. Gráfico de Línea: Evolución de Profit por Categoría (2.7)
profit_by_year_category = df.groupby(['YearOrderDate', 'Category'])['Profit'].sum().reset_index()
profit_by_year_category['YearOrderDate'] = profit_by_year_category['YearOrderDate'].astype(str)
fig_27 = px.line(profit_by_year_category, x='YearOrderDate', y='Profit', color='Category', markers=True)
fig_27.update_layout(showlegend=True, xaxis=dict(tickformat='d'))
fig_27.update_traces(showlegend=False)


# ---INICIO BLOQUE DE AJUSTES ---
# ---Ajustes de leyendas y colores antes del dashboard ---

# 1. Quitar leyendas y nombres que causan interacción en tortas
for fig in [fig_24, fig_25_orders, fig_25_profit, fig_26]:
    fig.update_traces(showlegend=False, name="")

# 2. Quitar leyendas de gráficos intermedios
fig_22.update_traces(showlegend=False)
fig_23.update_traces(showlegend=False)

# 3. Mantener leyenda solo en el gráfico final
fig_27.update_traces(showlegend=True)
fig_27.update_layout(
    legend=dict(
        orientation="v",      # horizontal
        yanchor="top",        # ancla superior
        y=1,                # un poco por debajo del gráfico
        xanchor="left",
        x=1.02
    )
)

# 4. Aplicar color azul uniforme a todos los gráficos simples
for fig in [fig_22, fig_23, fig_24, fig_25_orders, fig_25_profit, fig_26]:
    for trace in fig.data:
        # Gráficos de torta (pie) → propiedad 'marker.colors'
        if trace.type == "pie":
            trace.marker.colors = ['#4c78a8'] * len(trace.labels)
        # Gráficos de barras o líneas
        elif hasattr(trace, 'marker'):
            trace.marker.color = '#4c78a8'
        elif hasattr(trace, 'line'):
            trace.line.color = '#4c78a8'

# --- FIN BLOQUE AJUSTES ---


# --- II. CREACIÓN DE LA ESTRUCTURA DEL DASHBOARD (CORREGIDO) ---
dashboard = make_subplots(
    rows=5, cols=3,
    specs=[
        # Fila 1: KPIs (3 columnas separadas)
        [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],

        # Fila 2: Gráficos XY (2 columnas, la 3ra es NULL)
        [{"type": "xy"},None, {"type": "xy"}],

        # Fila 3: Gráficos Domain (2 columnas, la 3ra es NULL)
        [{"type": "domain"},None, {"type": "domain"}],

        # Fila 4: Gráficos Domain (2 columnas, la 3ra es NULL)
        [{"type": "domain"},None, {"type": "domain"}],

        # Fila 5: Gráfico XY (colspan: 2, ocupa las 2 primeras. Relleno con NULL para completar las 3)
        [None,{"type": "xy", "colspan": 2}, None],
    ],
    # Nombres de Subplots (10 títulos: 3 para Fila 1, 2 para Fila 2, etc.)
    subplot_titles=(
        "Órdenes Totales", "Facturación Total", "Ganancias Totales", # Fila 1
        "Evolución de Profit por Año", "Ventas Totales por Año", # Fila 2
        "Órdenes por Categoría", "Órdenes por Modo de Envío", # Fila 3
        "Órdenes por Región", "Ganancias por Región", # Fila 4
        "Evolución de Profit por Categoría" # Fila 5
    ),
    vertical_spacing=0.1 # Añadido para mejor separación visual
)


# --- III. AÑADIR COMPONENTES AL DASHBOARD (CORREGIDO) ---

# A. KPIs (Fila 1) - Insertamos cada KPI en su propia columna

kpi_values = [total_orders, total_sales, total_profit]
kpi_titles = ["Órdenes Totales", "Facturación Total", "Ganancias Totales"]
kpi_formats = [",.0f", ",.0f", ",.0f"]
kpi_prefix = ["", "$", "$"]

for i in range(3):
    # CAMBIO 2: Corrección de sintaxis y eliminación de dominio X innecesario
    dashboard.add_trace(go.Indicator(
        mode="number",
        value=kpi_values[i],
        # Ya no necesitamos el 'title' aquí, usamos el subplot_title de make_subplots
        number={'prefix': kpi_prefix[i], 'valueformat': kpi_formats[i], 'font': {'size': 20}},
        # Eliminamos la línea de dominio errónea
    ), row=1, col=i+1) # Insertamos en la columna correcta (1, 2, o 3)


# B. Gráficos (Filas 2, 3, 4, 5)

# Fila 2 (Columna 1 y 2)
for trace in fig_22.data: dashboard.add_trace(trace, row=2, col=1)
for trace in fig_23.data: dashboard.add_trace(trace, row=2, col=3)

# Fila 3 (Columna 1 y 2)
dashboard.add_trace(fig_24.data[0], row=3, col=1)
dashboard.add_trace(fig_26.data[0], row=3, col=3)

# Fila 4 (Columna 1 y 2)
dashboard.add_trace(fig_25_orders.data[0], row=4, col=1)
dashboard.add_trace(fig_25_profit.data[0], row=4, col=3)

# Fila 5 (Columna 1 - Colspan 2)
for trace in fig_27.data: dashboard.add_trace(trace, row=5, col=2)


# --- IV. AJUSTES FINALES ---

# CAMBIO 3: Ajustamos los márgenes para asegurar espacio
dashboard.update_layout(
    title_text="<b>MINI-DASHBOARD DE RENDIMIENTO DE SUPERSTORE</b>",
    height=1600,
    margin=dict(t=100, b=80, l=60, r=60), # t=100 da espacio para el título principal y la Fila 1
    showlegend=True,
    legend=dict(orientation="h", yanchor="top", y=0.04, xanchor="left", x=0.01)
)

# Ajustar los ejes X e Y de todos los subplots
# Fila 2 (Columna 1 y 2)
dashboard.update_xaxes(title_text='Año', row=2, col=1)
dashboard.update_xaxes(title_text='Año', row=2, col=2)
dashboard.update_yaxes(title_text='Beneficio ($)', row=2, col=1)
dashboard.update_yaxes(title_text='Ventas ($)', row=2, col=2)

# Fila 5 (Columna 1)
dashboard.update_xaxes(title_text='Año de la Orden', row=5, col=1)
dashboard.update_yaxes(title_text='Beneficio ($)', row=5, col=1)


dashboard.show()